## Ownershipe rules
- Each value in Rust has a variable that’s called its owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

In [2]:
let mut s = String::from("hello");
s.push_str(", world");
println!("{}", s);


hello, world


In [2]:
let s = String::from("hello");
s.push_str(", world");
println!("{}", s);


Error: cannot borrow `s` as mutable, as it is not declared as mutable

In [3]:
let mut s = "hello";
s.push_str(", world");
println!("{}", s);

Error: no method named `push_str` found for reference `&str` in the current scope

In [2]:
{
    let s = String::from("hello");
} // rust invoke `drop` function automatically here.   
println!("{}", s);

Error: cannot find value `s` in this scope

```
String {
    ptr: Heap
    len: number
    capacity: number
}
```

```
let s1 = String::from("hello");
let s2 = s1;
```

s1 and s2 do not share len and capacity, but share ptr.


In [3]:
let s1 = String::from("hello");
let s2 = s1;

println!("{}, world!", s1);

Error: borrow of moved value: `s1`

In [4]:
let s1 = String::from("hello");
let s2 = s1.clone();

println!("s1 = {}, s2 = {}", s1, s2);

s1 = hello, s2 = hello


In [6]:
let x = 5;
let y = x;

println!("x = {}, y = {}", x, y);

x = 5, y = 5


In [2]:

fn take_ownership (string: String) {
    println!("{}", string);
} // call drop and release backing memory

fn makes_copy(int: i32) {
    println!("{}", int);
} // call drop but do nothoing

let s = String::from("hello");
take_ownership(s); // moved

let x = 5;
makes_copy(x); // moved but still valid as x is `Copy`

println!("s = {}, x = {}", s, x);

Error: borrow of moved value: `s`

In [4]:
fn gives_ownership () -> String {
    let str = String::from("hello");
    str
}

fn takes_and_gives_back(str: String) -> String {
    str
}

let s1 = gives_ownership();
let s2 = String::from("hello");
let s3 = takes_and_gives_back(s2);

println!("s1 = {}, s2 = {}, s3 = {}", s1, s2, s3);


Error: borrow of moved value: `s2`

In [5]:
fn calc_length (s: String) -> (String, usize) {
    let length = s.len();
    (s, length)
}

let s1 = String::from("hello");
let (s2, len) = calc_length(s1); // s1 will be move and drop
println!("The length of '{}' is {}.", s2, len); 


The length of 'hello' is 5.


In [7]:
fn calc_length (s: &String) -> usize {
    s.len()
}

let s1 = String::from("hello");
let len = calc_length(&s1);
println!("The length of '{}' is {}.", s1, len);

The length of 'hello' is 5.


In [2]:
fn change (s: &String) {
    s.push_str(", world");
}
let s = String::from("hello");
change(&s);

Error: cannot borrow `*s` as mutable, as it is behind a `&` reference

In [6]:
fn change (s: &mut String) {
    s.push_str(", world");
}

let mut s = String::from("hello");
change(&mut s);
println!("{}", s);

hello, world


In [13]:
let mut s = String::from("hello");

{
    let a = &mut s;
    let b = &mut s;

    println!("{}, {}", a, b);
}

Error: cannot borrow `s` as mutable more than once at a time

In [12]:
let mut s = String::from("hello");

{
    {
        let a = &mut s;
        println!("a = {}", a);
    } // a becomes out of scope, then s is not borrowed by anyone.
    let b = &mut s;

    println!("b = {}", b);
}

a = hello
b = hello


()

In [3]:
let mut s = String::from("hello");

{
    let r1 = &s;
    let r2 = &s;
    println!("{}, {}", r1, r2);
}

hello, hello


()

In [15]:
let mut s = String::from("hello");

{
    let r1 = &s;
    let r2 = &s;
    let r3 = &mut s;
    println!("{}, {}, {}", r1, r2, r3);
}

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

In [17]:
fn dangle () -> &String {
    let s = String::from("hello");
    &s
} // `s` will be dropped

let s2 = dangle(); // but `s2` wants to see `s`
println!("{}", s2);

Error: missing lifetime specifier

In [2]:
fn dangle () -> String {
    let s = String::from("hello");
    s
}

let s2 = dangle(); // heap of `s` will be owned by `s2`
println!("{}", s2);

hello


In [4]:
// naive implementation of finding words

fn first_word(s: &String) -> usize {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return i;
        }
    }

    s.len()
}

let s = String::from("Hello world!");
println!("{}", first_word(&s));


5


In [7]:
{
    let s = String::from("Hello world!");
    println!("{}", &s[2..6]);
    println!("{}", &s[..7]);
    println!("{}", &s[4..]);
    println!("{}", &s[..]);
}

llo 
Hello w
o world!
Hello world!


()

In [6]:
{
    let s = String::from("Hello world!");
    println!("{}", &s[2..15]);
}

thread '<unnamed>' panicked at 'byte index 15 is out of bounds of `Hello world!`', src/lib.rs:144:21
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::str::slice_error_fail
   3: _run_user_code_4
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [10]:
// can't split multibyte char
{
    let s = String::from("Hello ��");
    println!("{}", &s[8..9]);
}

thread '<unnamed>' panicked at 'byte index 8 is not a char boundary; it is inside '�' (bytes 6..9) of `Hello ��`', src/lib.rs:40:21
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::str::slice_error_fail
   3: _run_user_code_6
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [13]:
// return sliced string
fn first_word(s: &str) -> &str {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }

    &s[..]
}

{
    let my_string = String::from("hello world");

    // first_wordは`String`のスライスに対して機能する
    let word = first_word(&my_string[..]);

    let my_string_literal = "hello world";

    // first_wordは文字列リテラルのスライスに対して機能する
    let word = first_word(&my_string_literal[..]);

    // 文字列リテラルは、すでに文字列スライス*な*ので、
    // スライス記法なしでも機能するのだ！
    let word = first_word(my_string_literal);
}


()